In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from typing import List

In [67]:
dataset = pd.read_csv('pornhub.com-db-shrinked1-0-fixed.csv', sep="|")

In [11]:
dataset.columns

Index(['embed', 'thumbnail', 'thumbnail2', 'title', 'tags', 'categories',
       'pornstars', 'duration', 'views', 'likes', 'dislikes', 'thumbnail3',
       'thumbnail4'],
      dtype='object')

In [68]:
dataset['categories'] = dataset.categories.str.split(';')
dataset['pornstars'] = dataset.pornstars.str.split(';')

In [69]:
dataset.loc[dataset['categories'].isnull()] = dataset.loc[dataset['categories'].isnull()].apply(lambda x: [])
dataset.loc[dataset['pornstars'].isnull()] = dataset.loc[dataset['pornstars'].isnull()].apply(lambda x: [])

In [80]:
dataset['query'] = dataset['categories'] + dataset['pornstars']
dataset['query'] = dataset['query'].map(lambda x: str(f"\"{str(x)}\""))

In [93]:
def recommendation(dataset: pd.DataFrame, minDuration: int, maxDuration: int, minViews: int, pornstars: List[str], categories: List[str]) -> pd.DataFrame:
    dataset = dataset[(dataset['duration'] >= minDuration) & (dataset['duration'] <= maxDuration) & (dataset['views'] >= minViews)]
    tf_model = TfidfVectorizer(max_features=5000, max_df=0.25, min_df=0.01, stop_words='english')
    tf_matrix = tf_model.fit_transform(dataset['query']).todense()
    tf_df = pd.DataFrame(tf_matrix)
    tf_df.columns = sorted(tf_model.vocabulary_)
    nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
    nn.fit(tf_matrix)
    new = tf_model.transform(pornstars + categories)
    result = nn.kneighbors(new.todense(), return_distance=False)
    prediction = []
    for i in result[1]:
        info = dataset.iloc[i]
        prediction.append({'embed': info['embed'], 'title': info['title'], 'duration': info['duration'],\
             'views': info['views'], 'categories': info['categories'], 'pornstars': info['pornstars'], 'tags': info['tags'],\
                'likes': info['likes'], 'dislikes': info['dislikes'],'thumbnail': info['thumbnail4'].split(';')})
    return prediction

In [94]:
recommendation(dataset, minDuration=100, maxDuration=100000, minViews=100, pornstars=[], categories=['Masturbation', 'Toys'])

C:\Users\bigma\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\bigma\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


[{'embed': '<iframe src="https://www.pornhub.com/embed/2101713069" frameborder="0" height="481" width="608" scrolling="no"></iframe>',
  'title': 'Jasmine Jae takes a hard pounding LIVE - she just loves the cock!',
  'duration': 450.0,
  'views': 222188.0,
  'categories': ['Big Tits', 'Toys', 'Pornstar', 'British'],
  'pornstars': ['Jasmine Jae'],
  'tags': 'adult-toys;sex-toy;69;hardcore;babe;saliva;killergram;anal-play;dildo;toys;vibrator;fishnets;atm;big-tits;busty;live-sex-show;fake-tits;british',
  'likes': 244.0,
  'dislikes': 62.0,
  'thumbnail': ['https://di.phncdn.com/videos/201406/06/27769761/original/(m=eaAaGwObaaaa)(mh=EblnxC-sArGvXqrG)1.jpg',
   'https://di.phncdn.com/videos/201406/06/27769761/original/(m=eaAaGwObaaaa)(mh=EblnxC-sArGvXqrG)2.jpg',
   'https://di.phncdn.com/videos/201406/06/27769761/original/(m=eaAaGwObaaaa)(mh=EblnxC-sArGvXqrG)3.jpg',
   'https://di.phncdn.com/videos/201406/06/27769761/original/(m=eaAaGwObaaaa)(mh=EblnxC-sArGvXqrG)4.jpg',
   'https://di.phn